Este repositorio contiene la configuración de un servidor de monitoreo climático para cámaras de frío, utilizando Docker para la orquestación de servicios y n8n como motor de flujos.

Primero partimos con n8n. Una vez creada nuestra cuenta y situados en el inicio de la plataforma, iniciamos un nodo de PostgreSQL del tipo Execute Query. Aquí es donde crearemos una tabla utilizando el comando que se muestra a continuación; esto servirá para que la base de datos pueda recibir y almacenar la información que envíe el ESP32.

Nota: Dentro del nodo, deben configurar una credencial con los datos específicos de su base de datos para establecer la conexión.

In [ ]:
CREATE TABLE lecturas_sensor (
    id SERIAL PRIMARY KEY,
    sensor_id VARCHAR(50),
    temp_interior DECIMAL(5,2),
    temp_exterior DECIMAL(5,2),
    fecha_registro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

Luego de crear la tabla, configuraremos un segundo workflow. Mientras que el anterior nos sirvió únicamente para la creación de la estructura, este nuevo flujo se encargará de recibir los datos del ESP32 e ingresarlos en la base de datos.

Configuración del Webhook: Primero, crearemos un nodo de tipo Webhook. Lo configuraremos con el método HTTP en POST y, en el apartado de path, asignaremos una ruta clara (en mi caso utilicé lecturas.camara). Este nodo generará la URL a la que el ESP32 enviará los datos.

Inserción en la Base de Datos: Segundo, crearemos un nodo de PostgreSQL con la operación Insert rows in a table. Aquí seleccionaremos la tabla correspondiente (asegúrate de usar el nombre exacto que definiste anteriormente; en mi caso es lecturas_sensor).

In [ ]:
Ahora realizaremos una prueba técnica para verificar que el sistema funciona correctamente. Para esto, utilizaremos un comando en la terminal que simula el envío de datos desde el ESP32 hacia nuestro servidor.

In [ ]:
curl -X POST http://192.168.100.53:5678/webhook-test/lecturas-camara ^
-H "Content-Type: application/json" ^
-d "{\"sensor_id\": \"test_pc\", \"temp_interior\": -5.5, \"temp_exterior\": 21.0}"

Si el proceso funcionó correctamente, veremos cómo avanza el flujo en tiempo real. En el nodo de inserción de PostgreSQL, podremos mapear (arrastrar) los valores de temperatura_int, temperatura_ext y sensor_id.

Es importante destacar que debemos dejar vacíos los campos de id y fecha_registro (en mi caso); esto se hace con el fin de que la base de datos los genere automáticamente al recibir la información, asegurando que cada registro tenga su marca de tiempo y su identificador único de forma exacta.

Finalmente, para corroborar que los datos se guardaron exitosamente, podemos volver al primer workflow (el que usamos para crear la tabla) y cambiar la consulta SQL por la siguiente:

In [ ]:
SELECT*
FROM lecturas_sensor(nombre de la tabla)

Si puedes visualizar un resultado en la consulta, significa que tu base de datos ya está lista para recibir información de manera constante.